In [66]:
import time
import faker

import pandas as pd
import numpy as np

import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from tqdm import tqdm
import glob
import requests
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import  Pool

In [67]:
import re
import gcld3

In [198]:
from googletrans import Translator
from googletrans.client import Timeout

In [250]:
tqdm.pandas()

In [69]:
fake = faker.Faker(locale='en')

In [70]:
lang_detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=800)

## Clearing of all csv's, drop duplicates and full nans

In [166]:
clear_string = lambda x: re.sub(' +', ' ',
                                re.sub('https?:\/\/.+\s', ' [URL] ',
                                       re.sub('<.*?>', ' ', str(x))
                                       .replace("&nbsp;", "")
                                       .replace("&rsquo;", "'")
                                       .replace("&amp;", "&")
                                       .replace("\u202f", " ")
                                       .replace("\xa0", " ")
                                       .replace("&mdash;", "-")
                                       .replace('.', '. ')
                                       .replace('\t', ' ')
                                       .replace("•", "-")
                                       .replace("·", "-")
                                       .replace("●", "-")
                                       .replace('\n', ' ')
                                       ).replace('/', ' / ').replace('\\', ' \ ').replace('|', ' | ')
                                ).strip()

In [167]:
csv_files = glob.glob('../data/*/*.csv')
len(csv_files)

12

In [168]:
for file in tqdm(csv_files):
       file_name = file.rsplit('.', 1)[0]
       compnany_name = file.split('/')[2]
       df = pd.read_csv(file, index_col=0, lineterminator='\n')

       df['title'] = df['title'].apply(clear_string)
       df['description'] = df['description'].fillna('N/A').apply(clear_string)
       df['responsibilities'] = df['responsibilities'].fillna('N/A').apply(clear_string)
       df['qualifications'] = df['qualifications'].fillna('N/A').apply(clear_string)

       df.replace('N / A', None, inplace=True)
       df.drop_duplicates(subset=['title', 'description', 'responsibilities', 'qualifications'], inplace=True)
       df.dropna(subset=['description', 'responsibilities', 'qualifications'], how='all', inplace=True)

       df.to_csv(file_name + '.csv', line_terminator='\n')
       df.to_excel(file_name + '.xlsx', engine='xlsxwriter')

100%|██████████| 12/12 [00:20<00:00,  1.73s/it]


## Merging in one DF

In [169]:
skip_companies_for_erge = ['alibaba']

In [170]:
data_frames = []
for file in tqdm(csv_files):
    file_name = file.rsplit('.', 1)[0]
    compnany_name = file.split('/')[2]

    df = pd.read_csv(file, index_col=0, lineterminator='\n')

    if compnany_name not in skip_companies_for_erge:
        data_frames.append(df[['title', 'description', 'responsibilities', 'qualifications', 'company' ,'publish_date']])

merged_dataset = pd.concat(data_frames)
merged_dataset.info()

100%|██████████| 12/12 [00:00<00:00, 12.04it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 38767 entries, 0 to 9042
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             38767 non-null  object
 1   description       35989 non-null  object
 2   responsibilities  30618 non-null  object
 3   qualifications    38750 non-null  object
 4   company           38767 non-null  object
 5   publish_date      37983 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [173]:
merged_dataset['company'].value_counts()

Amazon          8061
Apple           7799
Microsoft       7386
IBM             6455
Google          4566
Tencent         1839
Yandex           784
Sber             767
Huawei           594
YouTube          230
Tinkoff          154
Fitbit            91
DeepMind          40
Google Fiber       1
Name: company, dtype: int64

In [174]:
nan_replaced = merged_dataset.fillna('')
merged_dataset['merged_info'] = nan_replaced['title']+ ' ; ' + nan_replaced['description'] + ' ; ' + nan_replaced['responsibilities'] + ' | ' + nan_replaced['qualifications']

In [175]:
merged_dataset.sample(5)

,title,description,responsibilities,qualifications,company,publish_date,merged_info
5021,Graphics Verification Engineer,Do you love creating elegant solutions to high...,Develop verification plans in coordination wit...,Proficiency in C++ and / or SystemVerilog in a...,Apple,2022-04-25,Graphics Verification Engineer ; Do you love c...
8124,"Software Development Engineer, Customer Engage...",Job summary Do you want to be a part of the te...,NaN,Programming experience with at least one moder...,Amazon,2022-06-21,"Software Development Engineer, Customer Engage..."
795,Data Center Technician,Google isn't just a software company. The Hard...,Contribute to and lead efforts / projects in t...,Minimum qualifications: Experience working wit...,Google,2022-06-27 15:31:47.271000+00:00,Data Center Technician ; Google isn't just a s...
488,"Account Manager, Mid-Market Sales, Google Cust...",Businesses that partner with Google come in al...,Manage a portfolio of Google Ads accounts by u...,Minimum qualifications: Bachelor's degree or e...,Google,2022-06-29 16:32:04.797000+00:00,"Account Manager, Mid-Market Sales, Google Cust..."
3966,"ISE, Special Projects - Incubation Designer (C...",Are you passionate about user experience? Do y...,We are looking for an extraordinary hands-on d...,5+ years of professional work experienceAn exc...,Apple,2022-05-16,"ISE, Special Projects - Incubation Designer (C..."


## Translating to english of merged_info

### Google API

In [258]:
def google_translate_text(text: str, from_lang='auto', g_translator=Translator(user_agent=fake.chrome(), timeout=Timeout(10)), char_limit=5000):
    try:
        text = g_translator.translate(text[:char_limit], dest='en', src=from_lang).text
    except Exception as e:
        print(e)
    time.sleep(0.2) # 200 ms wait
    return text

### LibriTranslate API

In [260]:
LIBRE_TRANSLATE_API_KEY = "8fbcf6c1-5e6d-4391-956c-c854da647f9e"
LIBRE_API_URL = "https://libretranslate.com/translate"
# LIBRE_TRANSLATE_API_KEY = ""
# LIBRE_API_URL = "http://84.38.185.155:5000/translate"

In [270]:
def libri_translate_text(text: str, from_lang='auto', timeout=20, char_limit=5000):
    payload = {
        'q': text[:char_limit],
        'source': from_lang,
        'target': 'en',
        'format': 'text',
        'api_key': LIBRE_TRANSLATE_API_KEY
    }
    libre_headers = {
        'user-agent': fake.chrome(),
    }
    try:
        result = requests.post(LIBRE_API_URL, data=payload, headers=libre_headers, timeout=timeout).json()
        # print(result)
        return result['translatedText']
    except Exception as e:
        # print(e)
        return text

### Support methods

In [269]:
def translate_df_row(row: pd.Series, nan_symbol='N / A'):
    original_lang = lang_detector.FindLanguage(row['merged_info'][:200]).language
    # original_lang = lang_detector.FindLanguage(row['title']).language
    if original_lang != 'en':
        row['merged_info'] = google_translate_text(row['merged_info'], from_lang='auto')
        row['title'] = row['merged_info'].split(';', maxsplit=1)[0]
        # row['merged_info'] = libri_translate_text(row['merged_info'], from_lang=original_lang, timeout=35)
        # row['title'] = libri_translate_text(row['title'], from_lang=original_lang)
        # if row['description'] != nan_symbol:
        #     row['description'] = libri_translate_text(row['description'], from_lang=original_lang)
        # if row['responsibilities'] != nan_symbol:
        #     row['responsibilities'] = libri_translate_text(row['responsibilities'], from_lang=original_lang)
        # if row['qualifications'] != nan_symbol:
        #     row['qualifications'] = libri_translate_text(row['qualifications'], from_lang=original_lang)
    return row

In [266]:
def df_parallel_apply(df, func, n_cores=4, axis=1):
    df_split = np.array_split(df, n_cores)
    with ThreadPoolExecutor(max_workers=n_cores) as executor:
        futures = [executor.submit(lambda: part.apply(func, axis=axis)) for part in df_split]
        executor.shutdown(wait=True)
    transformed_parts = [f.result() for f in futures]
    return pd.concat(transformed_parts)

In [276]:
%%time
test_text = 'Первые впечатления Переночевав в гостинице в Гуаякиле, мы сели к агенту в машину и поехали на судно в Пуэрто Боливар. Доехали вопреки ожиданиям быстро, примерно за 3-4 часа. Погода была пасмурная и даже не смотря на то, что мы находимся недалеко от экватора, было прохладно. Почти все время, пока мы ехали, по обе стороны дороги были банановые плантации, но все равно в голове не укладывается: эти бананы грузят на суда в нескольких портах Эквадора десятками тысяч тонн каждый день, круглый год. Это ж несчастные бананы должны расти быстрее чем грибы. Дороги. Дороги в Эквадоре практически идеальные, хотя населенные пункты выглядят очень бедно. На дорогах много интересных машин, например очень много грузовиков - древних Фордов, которые я никогда раньше не видел. А еще несколько раз на глаза попадались старенькие Жигули :) А еще если кого-то обгоняешь и есть встречная машина, она обязательно включает фары. На больших машинах - грузовиках и автобусах, обязательно красуется местный тюнинг: машины разукрашенные, либо в наклейках, и обязательно везде огромное множество светодиодов, как будто новогодние елки едут и переливаются всеми цветами. Судно. На первый взгляд судно неплохое, в относительно хорошем состоянии, хотя и 92 года постройки. Экипаж 19 человек - 11 русских и 8 филиппинцев, включая повара. Говорят, периодически становится тоскливо от егошних кулинарных изысков. Филиппинцы здесь рядовой состав, за ними постоянно нужно следить чтобы не натворили чего, среди них только один матрос по-настоящему ответственный и с руками из нужного места, все понимает с полуслова. Остальные - типичные Равшаны да Джамшуты. А еще один из них - гомосек О___о, в добавок к этому он опасный человек, в том плане, что легко впадает в состояние ступора и отключает мозг: был случай как он закрыл одного матроса в трюме, тот орал и тарабанил внутри, это заметил боцман, начал орать на этого персонажа, который, в свою очередь испуганно выпучив глаза, трясущимися руками продолжал закручивать барашки. В итоге боцман его отодвинул и выпустил матроса из трюма. Общение на английском языке, но из-за акцента не всегда с первого раз понятно что филиппинцы говорят, особенно по рации. Напимер, говорит он тебе: Бикарпуль! Бикарпуль! А потом, когда уже поздно, выясняется что это было "Be careful!" Работа. Сразу, как только мы заселились, я не успел разложить вещи, как в мою голову ворвался такой поток информации, что ни в сказке сказать, ни топором не вырубить. Во-первых, на судне абсолютно все бумаги - мануалы, журналы, и так далее - все на английском языке. Даже блокнотик, в который записываются отчеты по грузовым операциям - и тот на английском. Бумаги... ооооо... Их тысячи, лежат в сотнях папок, плюс огромное количество документов на компьютерах. Это мне просто разорвало мозг в клочья, потому что с этим объемом информации надо ознакомиться и научиться работать в кротчайшие сроки. Постоянная беготня, постоянная суета, совсем не легко. А также надо как можно быстрее разобраться со всем оборудованием на мостике, а там его мама не горюй. В общем, пока что, свободного времени нет вообще. Абсолютно. Только ночью с 00:00 до 06:00 можно поспать. Но это продлится не долго, буквально 1-2 недели, потом океанский переход до Европы, можно будет уже спокойно стоять вахты, а в свободное время читать книги компании Seatrade, на случай если в Европе придет проверка и будет задавать вопросы. Ну и немного о приятном. Неплохая одноместная каюта. Внутри несколько шкафов и полок, удобная кровать, койка напередохнуть, стол, стул, умывальник и внутрисудовой телефон. Также выдали 2 белых офицерских комбинезона с символикой компании, каску и персональную рацию. В моем распоряжении офицерский душ со стиральной машинкой и офицерская столовая. Во время швартовых операций мое место на мостике. Хотя и матросы на палубе не сильно напрягаются - там установлены гидравлические лебедки, не надо швартовные концы тягать вручную. На этом пока все, фоток пока что нет, потому что просто некогда этим заниматься. Даже этот текст я пишу потому что просто появилось свободных 2 часа, отпустили отдохнуть, так как впереди 6-часовая вахта, а сразу после нее отшвартовка и идем в Гуаякиль, а во время отшвартовки и пока лоцман на судне - мое место на мостике. Так что, предстоит основательно заколебаться. Возле входа в канал находится якорная стоянка, где куча судов стоит и ждет своей очереди на проход. Но, нам дали добро на проход практически без ожидания и постояв на якоре всего три часа, приехал лоцман, скомандовал "dead slow ahead" (самый малый вперед), я перевел ручку машинного телеграфа, похожую на автомобильный рычаг автоматической коробки передач в указанное положение, на что телеграф отозвался громким сигналом. Стрелка тахометра поползла вверх, судно слегка задрожало и мы медленно но верно начали набирать скорость и двигаться в сторону канала. Вход в канал со стороны океана выделен буями - десятки буев, выстроенные в две линии показывают единственный верный путь, выгладит грандиозно. А справа, за пальмами и дорогими виллами со стоящими рядом с ними роскошными яхтами, простирется огромный город небоскребов - Панама. А через десять минут открылось еще одно не менее грандиозное зрелище - строительство новых шлюзов, которые будут пропускать суда бОльших размеров, чем нынешние шлюзы. Но тут впереди из-за поворота показалась огромная стена, которая двигалась в нашу сторону. Навстречу нам шло судно-автомобилевоз Green Cove и так как канал - место совсем не широкое, автомобилевоз прошел от нас всего в 50 метрах, непривычно было стоя на мостике наблюдать столь близкое расхождение. А вот и буксиры. Это значит что мы подходим к шлюзам. Только мы подошли к шлюзу, как тут же нас привязали к резвым локомотивам - Мулам. Дальше все зависело только от них. 4 Мула держали нас с разных сторон и уже они руководили процессом, они нас резво разгоняли и останавливали в нужном месте и терпеливо ждали пока шлюз не наполнится водой. Так как наше судно не очень широкое, проблем с заходом в шлюз не было, а по длине получилось так, что за нами сзади поместилось еще одно небольшое судно. Правда мы оказались более быстроходными и пока мы шли от тихоокеанских до атлантических шлюзов по каналу и озеру Гатун, это судно безнадежно отстало, но нам пришлось его ждать :('
print(test_text.split(' ').__len__(), test_text[:5000].split(' ').__len__())
google_translate_text(test_text, from_lang='auto').split(' ').__len__()

971 766
CPU times: user 1.45 s, sys: 396 ms, total: 1.85 s
Wall time: 1.39 s


991

In [241]:
%%time
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(lambda: google_translate_text(test_text[:5000], from_lang='ru')) for i in range(2)]
    executor.shutdown(wait=True)
transformed_parts = [f.result() for f in futures]
[x.split(' ').__len__() for x in transformed_parts]

[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2635)


KeyboardInterrupt: 

### Transalting & cleaning process

In [272]:
%%time
merged_dataset[merged_dataset['company'] == 'Sber'][:5].progress_apply(translate_df_row, axis=1)

100%|██████████| 5/5 [00:02<00:00,  2.36it/s]

CPU times: user 2.13 s, sys: 617 ms, total: 2.75 s
Wall time: 2.15 s


,title,description,responsibilities,qualifications,company,publish_date,merged_info
0,Финансовый консультант (Доставка банковских пр...,NaN,Финансовый консультант – это новая перспективн...,"Ты нам подходишь, если: - Ранее занимался дост...",Sber,2022-01-12 00:00:00,Financial consultant (delivery of banking prod...
1,Руководитель направления / Лидер кластера Дого...,NaN,определение видения развития сервисов: digital...,высшее (экономическое) образование; наличие оп...,Sber,2021-12-03 00:00:00,Head of the Direct / Cluster Leader Contract S...
2,Стажер автотестировщик (Инвестиции),Сбер - это крупнейшая цифровая экосистема. Тех...,"Тестирование веб приложений, мобильных приложе...","Практический опыт функционального, интеграцион...",Sber,2021-12-09 00:00:00,Traineer traineer (investment); Sber is the la...
3,Сетевой эксперт (Remote Access),NaN,Развитие сервиса Remote access в банке; Админи...,Высшее техническое образование; Опыт работы в ...,Sber,2022-02-22 00:00:00,Network expert (Remote Access); ; Development ...
4,QA automation engineer (проект MLStorageр),О компании: АО «Сбербанк-Технологии» - российс...,Мы ценим: Готовность работать самостоятельно и...,"Мы ищем человека, который: Имеет опыт работы т...",Sber,2022-02-28 00:00:00,QA Automation Engineer (project mlstorager); A...


In [273]:
%%time
translated_merged_dataset = merged_dataset.progress_apply(translate_df_row, axis=1)

 43%|████▎     | 16491/38767 [53:06<6:51:29,  1.11s/it] 

The read operation timed out


100%|██████████| 38767/38767 [2:24:43<00:00,  4.46it/s]      

CPU times: user 1h 38min 23s, sys: 30min 57s, total: 2h 9min 20s
Wall time: 2h 24min 43s


In [275]:
translated_merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38767 entries, 0 to 9042
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             38767 non-null  object
 1   description       35989 non-null  object
 2   responsibilities  30618 non-null  object
 3   qualifications    38750 non-null  object
 4   company           38767 non-null  object
 5   publish_date      37983 non-null  object
 6   merged_info       38767 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [291]:
translated_merged_dataset['merged_info'].progress_apply(lambda x: lang_detector.FindLanguage(x[:200]).language).value_counts()

100%|██████████| 38767/38767 [00:18<00:00, 2058.99it/s]


en         38500
xh            38
zu            34
ja            28
mg            28
no            11
fi            11
lb            11
da            11
ga             8
zh             7
fr             6
de             6
cy             4
et             4
so             4
af             4
la             4
it             4
pt             3
ko             3
ky             3
sv             3
mr             2
zh-Latn        2
sn             2
nl             2
ca             2
vi             2
fil            2
mt             2
el-Latn        1
mi             1
su             1
es             1
sr             1
gd             1
haw            1
is             1
gl             1
co             1
ru             1
ru-Latn        1
ne             1
jv             1
cs             1
sm             1
Name: merged_info, dtype: int64

In [295]:
translated_merged_dataset[translated_merged_dataset['company'] == 'Huawei'].sample(10)

,title,description,responsibilities,qualifications,company,publish_date,merged_info
767,行业云计算解决方案专家,NaN,"""1. 研究政务、税务、医疗、交通、教育等政企行业客户对大数据咨询、管理和分析等服务的需求，...","业务技能要求： ""1. 精通大数据分布式系统（Hadoop、Spark、MPPDB等）的架构...",Huawei,2020-09-28 00:00:00+08:00,"Industry cloud computing solutions experts; ""1..."
40,渠道销售专员,NaN,参与所属领域的合作伙伴规划，在指导下负责某个领域合作伙伴拓展、合作伙伴联合营销活动，实现所负...,业务技能要求： 了解、执行业务范围内MPR、OTO等流程；具有伙伴合作能力、团队管理能力、营...,Huawei,2022-06-25 00:00:00+08:00,Channel sales commissioner;; partnership plann...
690,云数据中心站点经理,NaN,全面负责国内或海外数据中心站点的运营管理、项目运作支持、机房服务、数据中心经营等职责，保障云...,业务技能要求： 1、具备数据中心运营服务的管理能力，能结合实际情况均衡考虑机房服务质量、效率...,Huawei,2022-01-24 00:00:00+08:00,Cloud data center site manager;; comprehensive...
375,金融解决方案专家,NaN,1、深入洞察金融行业ICT技术趋势，负责金融行业解决方案销售策略制定； 2、负责技术层的hi...,业务技能要求： 1、英语可作为工作语言； 2、成就导向，冲劲足，接受海外出差； 3、Base...,Huawei,2022-05-12 00:00:00+08:00,Financial solution experts; 1. In -depth insig...
419,生产计划高级工程师,NaN,1、负责与所覆盖区域做好供需协同，根据区域需求组织制定和实施供应中心计划，管理供需风 险，确...,业务技能要求： 1、具备较强的逻辑思维能力、数据分析能力、沟通能力和跨部门协调能力。 2、具...,Huawei,2022-04-29 00:00:00+08:00,Senior engineer of production plan; 1. Respons...
369,订单管理工程师,NaN,标准职责 供应中心面向区域中大型代表处 / 国家端到端订单履行方案的执行人，为所负责国家 /...,业务技能要求： 技能： 1、订单需求管理：备发货条件管理；订单计划制定与变更管理；订单状态管...,Huawei,2022-05-13 00:00:00+08:00,Order management engineer;; Standard responsib...
342,物流专员,NaN,标准职责 面向区域 / BG / 项目提供物流服务，适配物流解决方案，协调解决业务异常，协同...,业务技能要求： 熟练运用业务系统和管理工具，熟悉数据分析和挖掘工具、方法；具备物流履行与业务...,Huawei,2022-05-19 00:00:00+08:00,Logistics commissioner;; standard responsibili...
231,【专业类】质量控制工程师,NaN,1、基于产品早期质量策划，通过新产品试制导入验证，进行新产品质量验证，达成新产品上市质量目标...,知识： 1、熟悉产品知识、产品工艺知识、新产品导入流程，质量问题处理及闭环，质量控制等。 技...,Huawei,2022-06-04 00:00:00+08:00,[Professional Category] Quality Control Engine...
564,基建项目管理专家（建筑设计 / 精装 / 土建 / 机电 / 幕墙 / 园林 / 项目管理等）,NaN,清晰理解项目定位和要求，理解用户需求并转换为设计语言。 面向中小型项目，从可运维、可管理和业...,业务技能要求： 1、具有扎实的专业管理能力（建筑 / 土建结构 / 幕墙 / 强电 / 弱电...,Huawei,2022-04-11 00:00:00+08:00,Infrastructure project management experts (arc...
98,高级产品经理（专业类）,NaN,1、负责数据分析、数字化作战和跨产品拓展支持，对洞察营销数据及跨产品项目运作负责。 2、依据...,业务技能要求： 熟练掌握ICT行业市场分析、运营商客户需求洞察、产品解决方案宣讲、项目投标和...,Huawei,2022-06-15 00:00:00+08:00,Senior product manager (professional category)...


In [296]:
remove_non_lat = lambda x: re.sub(' +', ' ', re.sub(r'[^a-zA-Z\d\s\',.:!?()\[\]&{};-]', ' ', x)).strip()

In [297]:
translated_merged_dataset['merged_info'] = translated_merged_dataset['merged_info'].progress_apply(remove_non_lat)

100%|██████████| 38767/38767 [00:10<00:00, 3719.52it/s]


In [322]:
translated_merged_dataset.to_csv('../results/translated_merged_dataset.csv', line_terminator='\n')
translated_merged_dataset.to_excel('../results/translated_merged_dataset.xlsx', engine='xlsxwriter')